In [133]:
# key_words = ['account', 'member', 'access', 'email', 'address',
#              'update', 'price', 'market', 'online', 'information',
#              'work', 'credit', 'response', 'offer', 'transaction',
#              'agreement', 'offer', 'registration', 'person', 'system',
#              'system', 'process', 'service', 'request', 'message']

key_words = ['account', 'member', 'access', 'email', 'address',
             'update', 'price', 'market', 'online', 'information',
             'work', 'credit', 'response', 'offer', 'transaction',
             'agreement', 'offer', 'registration']

len(key_words)

18

In [134]:
class MailContentsFeaturize(object):
    def __init__(self, content):
        self.content = content
        
    # 提取特征方法
    def contain_key(self, key):
        if key in self.content:
            return 1
        else:
            return 0
    
    def run(self):
        data = {}
        for word in key_words :
            data[word] = self.contain_key(word)
        return data
    
    def get_train_data(self):
        data = []
        for word in key_words :
            data.append(self.contain_key(word))
        return data

In [136]:
no_head_train_path_0 = '../../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../../data/phishing/IWSPA-AP-traindata/legit/'
no_head_test_path = '../../data/phishing/IWSPA-APTestData/testdata_noheaders/'

no_head_test_wc_normal_path = "../../test_data/from_weicheng/normal_txt/"
no_head_test_wc_phishing_path = "../../test_data/from_weicheng/phishing_txt/"

no_head_test_jose_phishing_path = "../../test_data/jose-phishing/"

print(no_head_train_path_0)
print(no_head_train_path_1)
print(no_head_test_path)
print(no_head_test_wc_normal_path)
print(no_head_test_wc_phishing_path)

../../data/phishing/IWSPA-AP-traindata/phish/
../../data/phishing/IWSPA-AP-traindata/legit/
../../data/phishing/IWSPA-APTestData/testdata_noheaders/
../../test_data/from_weicheng/normal_txt/
../../test_data/from_weicheng/phishing_txt/


In [137]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [138]:
import os

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        if text_file.endswith(".txt"):
            file_path = os.path.join(path, text_file)
            # print(file_path)
            read_file = open(file_path,'r+')
            read_text = read_file.read()
            read_file.close()
            cleaned_text = clean_text(read_text)
            # append text
            # text_list.append(cleaned_text)

            featurized = MailContentsFeaturize(cleaned_text)
            # append features
            # text_list.append(featurized.run())
            text_list.append(featurized.get_train_data())
    return text_list, files

In [139]:
# 读取数据
no_head_train_0, temp = get_data(no_head_train_path_0)
no_head_train_1, temp = get_data(no_head_train_path_1)
no_head_test, no_head_files = get_data(no_head_test_path)

no_head_test_wc_normal, normal_wc_test_file = get_data(no_head_test_wc_normal_path)
no_head_test_wc_phishing, phishing_wc_test_file = get_data(no_head_test_wc_phishing_path)

no_head_test_jose_phishing, jose_phishing_test_file = get_data(no_head_test_jose_phishing_path)

print(no_head_train_0.__len__())
print(no_head_train_0[0])

print(no_head_train_1.__len__())
print(no_head_train_1[0])

print(no_head_test.__len__())
print(no_head_test_wc_normal.__len__())
print(no_head_test_wc_phishing.__len__())
print(no_head_test_jose_phishing.__len__())

629
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5092
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4300
20
16
10148


In [140]:
# label: 0 -> phish ; 1 -> normal

no_head_train = no_head_train_0 + no_head_train_1
no_head_labels_train = ([0] * len(no_head_train_0)) + ([1] * len(no_head_train_1))
print(no_head_train.__len__())
print(no_head_labels_train.__len__())

no_head_labels_test = [1] * len(no_head_test)

no_head_label_test_wc_normal = [1] * len(no_head_test_wc_normal)
no_head_label_test_wc_phishing = [0] * len(no_head_test_wc_phishing)

no_head_label_test_jose_phishing = [0] * len(no_head_test_jose_phishing)

print(no_head_label_test_wc_normal.__len__())
print(no_head_label_test_wc_phishing.__len__())

5721
5721
20
16


In [141]:
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# tf_vectorizer = CountVectorizer()
# X = tf_vectorizer.fit_transform(no_head_train)
# print ('#total words', np.matrix.sum(X.todense()))
# print ('#unique words',len(set(tf_vectorizer.get_feature_names())))

In [142]:
shuffled_indices = np.random.permutation(len(no_head_labels_train))
train_data = np.array(no_head_train)[shuffled_indices]
train_data = train_data.tolist()
train_label = np.array(no_head_labels_train)[shuffled_indices]
train_label = train_label.tolist()

shuffled_indices_test = np.random.permutation(len(no_head_labels_test))
test_data = np.array(no_head_test)[shuffled_indices_test]
test_data = test_data.tolist()
test_label = np.array(no_head_labels_test)[shuffled_indices_test]
test_label = test_label.tolist()

shuffled_indices_test_wc_normal = np.random.permutation(len(no_head_label_test_wc_normal))
test_data_wc_normal = np.array(no_head_test_wc_normal)[shuffled_indices_test_wc_normal]
test_data_wc_normal = test_data_wc_normal.tolist()
test_label_wc_normal = np.array(no_head_label_test_wc_normal)[shuffled_indices_test_wc_normal]
test_label_wc_normal = test_label_wc_normal.tolist()
#
shuffled_indices_test_wc_phishing = np.random.permutation(len(no_head_label_test_wc_phishing))
test_data_wc_phishing = np.array(no_head_test_wc_phishing)[shuffled_indices_test_wc_phishing]
test_data_wc_phishing = test_data_wc_phishing.tolist()
test_label_wc_phishing = np.array(no_head_label_test_wc_phishing)[shuffled_indices_test_wc_phishing]
test_label_wc_phishing = test_label_wc_phishing.tolist()
## jose-phishing test data
shuffled_indices_test_jose_phishing = np.random.permutation(len(no_head_label_test_jose_phishing))
test_data_jose_phishing = np.array(no_head_test_jose_phishing)[shuffled_indices_test_jose_phishing]
test_data_jose_phishing = test_data_jose_phishing.tolist()
test_label_jose_phishing = np.array(no_head_label_test_jose_phishing)[shuffled_indices_test_jose_phishing]
test_label_jose_phishing = test_label_jose_phishing.tolist()

In [143]:
temp_train_data = train_data[0:int(0.8*len(train_data))]
temp_train_label = train_label[0:int(0.8*len(train_label))]
temp_test_data = train_data[int(0.8*len(train_data)):]
temp_test_labels = train_label[int(0.8*len(train_label)):]

In [144]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [145]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression


In [166]:
#Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(temp_train_data,temp_train_label)

dt_predictions = dt_model.predict(temp_test_data)
accuracy_score(temp_test_labels,dt_predictions)
print(confusion_matrix(temp_test_labels,dt_predictions))

0.9222707423580786

In [147]:
dt_predictions = dt_model.predict(test_data)
accuracy_score(test_label, dt_predictions)
print(confusion_matrix(test_label,dt_predictions))

0.9167441860465116

In [148]:
dt_predictions = dt_model.predict(test_data_wc_normal)
accuracy_score(test_label_wc_normal, dt_predictions)
print(confusion_matrix(test_label_wc_normal,dt_predictions))

0.6

In [149]:
dt_predictions = dt_model.predict(test_data_wc_phishing)
accuracy_score(test_label_wc_phishing, dt_predictions)
print(confusion_matrix(test_label_wc_phishing,dt_predictions))

0.25

In [152]:
dt_predictions = dt_model.predict(test_data_jose_phishing)
accuracy_score(test_label_jose_phishing, dt_predictions)
print(confusion_matrix(test_label_jose_phishing,dt_predictions))


0.556267244777296

In [154]:

#Random Forest
rfc = RandomForestClassifier()
rfc.fit(temp_train_data, temp_train_label)

dt_predictions = rfc.predict(temp_test_data)
accuracy_score(temp_test_labels,dt_predictions)

0.9222707423580786

In [155]:
dt_predictions = rfc.predict(test_data)
accuracy_score(test_label, dt_predictions)

0.9206976744186046

In [156]:
dt_predictions = rfc.predict(test_data_wc_normal)
accuracy_score(test_label_wc_normal, dt_predictions)

0.6

In [157]:
dt_predictions = rfc.predict(test_data_wc_phishing)
accuracy_score(test_label_wc_phishing, dt_predictions)

0.1875

In [158]:
dt_predictions = rfc.predict(test_data_jose_phishing)
accuracy_score(test_label_jose_phishing, dt_predictions)

0.5300551832873472

In [121]:
print("Random Forest")
print(confusion_matrix(temp_test_labels,dt_predictions))

Random Forest


ValueError: Found input variables with inconsistent numbers of samples: [1145, 16]

In [159]:
#Logistic Regression
log_model = LogisticRegression()
log_model.fit(temp_train_data, temp_train_label)

log_predictions = log_model.predict(temp_test_data)
accuracy_score(temp_test_labels,log_predictions)


0.9283842794759826

In [160]:
log_predictions = log_model.predict(test_data)
accuracy_score(test_label, log_predictions)

0.9227906976744186

In [161]:
dt_predictions = log_model.predict(test_data_wc_normal)
accuracy_score(test_label_wc_normal, dt_predictions)

0.55

In [162]:
dt_predictions = log_model.predict(test_data_wc_phishing)
accuracy_score(test_label_wc_phishing, dt_predictions)


0.25

In [163]:
dt_predictions = log_model.predict(test_data_jose_phishing)
accuracy_score(test_label_jose_phishing, dt_predictions)

0.5626724477729602

In [64]:
print("Logistic Regression")

print(confusion_matrix(temp_test_labels,log_predictions))


Logistic Regression


ValueError: Found input variables with inconsistent numbers of samples: [1145, 4300]